Unifying Datasets Remote
========

In [2]:
import pandas as pd
import numpy as np
from glob import glob

In [3]:
files = glob('../inmet_datasets/*.csv')

In [4]:
# replaces every ';' with ',' and ' ' with '_', and creates a new file

def rewrite_file(file_name):
    with open('../pre-processed_datasets/' + file_name.split('datasets/')[1], 'w') as new_file:
        with open(file_name, 'r+') as file:            
            for line in file:
                line = line.replace(';', ',')
                line = line.replace(' ', '_')
                line = line[: -2]
                
                new_file.write(line + '\n')
        
            file.close()
        
        new_file.close()
        

In [5]:
def merge_rows (dataset):
    """"
    Function responsible for normalizing the dataset
    
    It receives a pandas dataframe, and, for each row where the column 'Hora' is 1200,
    it takes the 'Precipitacao' and 'TempMinima' columns and puts its values on the
    row above, for it does not have such values (NaN value).
    """
    for index, row in dataset.iterrows():
        if row.loc['Hora'] == 1200:
            precipitacao = row.loc['Precipitacao']
            temp_minima = row.loc['TempMinima']

            dataset.at[index - 1, 'Precipitacao'] = precipitacao
            dataset.at[index - 1, 'TempMinima'] = temp_minima

In [18]:
def precipitation_rate(dataset, min_len):
    r0 = len(dataset[dataset['Precipitacao'] == 0])
    r1 = len(dataset[dataset['Precipitacao'] == 1])
    
    r0 = r0/len(dataset)
    r1 = r1/len(dataset)
    
    n0 = int(round(r0 * min_len))
    n1 = int(round(r1 * min_len))
    
    df0 = dataset[dataset['Precipitacao'] == 0].sample(n=n0)
    df1 = dataset[dataset['Precipitacao'] == 1].sample(n=n1)
    
    return pd.merge(df0, df1, on=['Estacao', 'Data', 'Hora', 'Precipitacao', 'TempMaxima', 'TempMinima', 'Insolacao', 'Evaporacao_Piche', 'Temp_Comp_Media', 'Umidade_Relativa_Media', 'Velocidade_do_Vento_Media'], how='outer')

In [7]:
dataset_min_len = []

for file in files:
    rewrite_file(file)
    
    csv_file = pd.read_csv('../pre-processed_datasets/' + file.split('datasets/')[1])
    
    dates = csv_file['Data'].unique()
    
    # deleting unique dates
    for date in dates:
        if (len(csv_file[csv_file['Data'] == str(date)]) < 2):
            csv_file = csv_file[csv_file['Data'] != str(date)]
    
    # enconding 'Precipitacao' into 0 or 1
    csv_file['Precipitacao'] = csv_file['Precipitacao'].apply(lambda x: 1 if x > 0 else 0)
    
    merge_rows(csv_file)
    
    csv_file.dropna(how='any', inplace=True)
    
    dataset_min_len.append(len(csv_file))
    
    # saving file
    csv_file.to_csv('../pre-processed_datasets/' + file.split('datasets/')[1], index=False)

In [13]:
unified = pd.DataFrame(columns=['Estacao', 'Data', 'Hora', 'Precipitacao', 'TempMaxima', 'TempMinima', 'Insolacao', 'Evaporacao_Piche', 'Temp_Comp_Media', 'Umidade_Relativa_Media', 'Velocidade_do_Vento_Media'])

In [19]:
dataset_min_len = np.amin(dataset_min_len)

for file in files:
    csv_file = pd.read_csv('../pre-processed_datasets/' + file.split('datasets/')[1])
    dataframe = precipitation_rate(csv_file, dataset_min_len)
    unified = pd.merge(unified, dataframe, on=['Estacao', 'Data', 'Hora', 'Precipitacao', 'TempMaxima', 'TempMinima', 'Insolacao', 'Evaporacao_Piche', 'Temp_Comp_Media', 'Umidade_Relativa_Media', 'Velocidade_do_Vento_Media'], how='outer')

In [23]:
unified.to_csv('./unified.csv', index=False)

In [ ]:
# backup function

# def unify_datasets():
#     for file in files:
#         file_name = '../pre-processed_dataset/' + file.split('datasets/')[1]
#         dataframe_file = pd.read_csv(file_name)
#         unified = pd.merge(unified, dataframe_file, on=['Estacao', 'Data', 'Hora', 'Precipitacao', 'TempMaxima', 'TempMinima', 'Insolacao', 'Evaporacao_Piche', 'Temp_Comp_Media', 'Umidade_Relativa_Media', 'Velocidade_do_Vento_Media'], how='outer')
#     unified.to_csv('../unified.csv', index=False)

In [14]:
print(dataset_min_len)

[28740, 36771, 31269, 40249, 30144, 31370, 30700, 31951, 30751, 12978, 18168, 28501, 33749, 34123, 31622, 16674, 30010, 33849, 29063, 38121, 30070, 38502, 21928, 29894, 40723, 36465, 28175, 28283, 39678, 29986, 30885, 35220, 28882, 21458, 33951, 27718, 30536, 31308, 37844, 42142, 27513, 24909, 32743, 21390, 37695, 20362, 30891, 39155, 29034, 27347, 35490, 37178, 39479, 38833, 39241, 34788, 30486, 27898, 35862, 41983, 32558, 38033, 32168, 23221, 28942, 33210, 35201, 30293, 31131, 29919, 33161, 26683, 32921, 24262, 26242, 35500, 34521, 18015, 40411, 36279, 29168, 22248, 33703, 38070, 24202, 36193, 40734, 34800, 30081, 37177, 23323, 25994, 31829, 41531, 31298, 41868, 27708, 13287, 28873, 38937, 29747, 28875, 37338, 15316, 34068, 35635, 31200, 34513, 27635, 18180, 16450, 33133, 37484, 33550, 29014, 26243, 26581, 34203, 32400, 27966, 39070, 28635, 33047, 38638, 24987, 21939, 36199, 25196, 25783, 33268, 17079, 23407, 36201, 30232, 38517, 26153, 36671, 35382, 37045, 41725, 35251, 29856, 38201

In [15]:
# smallest length

np.amin(dataset_min_len)

12978